In [8]:
import re

In [7]:
with open("../datasets/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20467
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Tokenize Data

In [20]:
# Identify the items we want to be separate tokens

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)

In [21]:
# Whitespace removal can reduce memory requirements and speed up training but should not be done for text that is sensitive to
# the structure of the text such as programming languages

preprocessed = [item for item in preprocessed if item.strip()]

In [22]:
preprocessed[:30]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

In [23]:
print(len(preprocessed))

4687


# Convert tokens into token IDs
- Vocabulary is constructed by listing the tokens into alphabetic order and then map each unique token to a unique integer as the ID
- We can think of this also as encoding.  LLM needs to decode in order to generate the output by reverse mapping ID to token

In [24]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [25]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [26]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [30]:
# Put this all together in a class that will have an encode method, and a decode method

class SimpleTokenizerV1:
    def __init__(self, vocab):
        # create vocab - mapping from tokens to token IDs (str to int)
        self.str_to_int = vocab
        # Reverse map of token ID to token (int to str)
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        # Split text on chars and remove whitespace
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        # Generate and assign token IDs
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    
    def decode(self, ids):
        # Convert token IDs back into tokens
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text
    

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know, "
            Mrs. Gisburn said with a pardonable pride.
"""

# Use the encode method on text and return token IDs
ids = tokenizer.encode(text)

print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 115, 754, 793, 7]


In [33]:
# Use the decode method to convert the token IDs back into text
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with a pardonable pride.'

In [ ]:
# Test tokenization on unseen text (won't work due to 'Hello' not being in the vocabulary)
new_text = "Hello, do you like tea?"
print(tokenizer.encode(new_text))


KeyError: 'Hello'

In [35]:
# Special context tokens 
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [37]:
len(vocab.items())

1132

In [39]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [41]:
# Extend class to handle end of sentence and unkown words

class SimpleTokenizerV2:
    def __init__(self, vocab):
        # create vocab - mapping from tokens to token IDs (str to int)
        self.str_to_int = vocab
        # Reverse map of token ID to token (int to str)
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        # Split text on chars and remove whitespace
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Replace unkown words with '<|unk|>' tokens
        preprocessed = [
                item if item in self.str_to_int
                else "<|unk|>" for item in preprocessed
        ]

        # Generate and assign token IDs
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    
    def decode(self, ids):
        # Convert token IDs back into tokens
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [42]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [43]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [44]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'